In [ ]:
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
from fiber.utils import Timer
import pickle

In [ ]:
#load Cohorts
Entire_Cohort_678k= pq.read_table('Merged_Cases_Controls_180.parquet').to_pandas()
Entire_Cohort_678k = Entire_Cohort_678k.reindex(sorted(Entire_Cohort_678k.columns), axis=1)
Entire_Cohort_678k.set_index('medical_record_number', inplace = True)

In [ ]:
for col in Entire_Cohort_678k:
    print (col)

In [ ]:
#check for old cases (131k) and controls(547k) 
Entire_Cohort_678k.loc[Entire_Cohort_678k['HT']== '1']

In [ ]:
#define only cases for master thesis (only cases with HT)
Cases_HT = Entire_Cohort_678k.loc[Entire_Cohort_678k['HT'] == '1']
Cases_HT

In [ ]:
for col in Cases_HT:
    print (col)

In [ ]:
#Cases_HT= Cases_HT.reset_index('medical_record_number')
check = Cases_HT.loc[Cases_HT['medical_record_number'] == '1000000506']
check['age_in_days']

In [ ]:
#add onset year
def add_onset_year(df_main):
    df_main['date_of_birth'] = pd.to_datetime(df_main['date_of_birth'])
    df_main['year_of_birth'] = pd.DatetimeIndex(df_main['date_of_birth']).year
    df_main['date_of_birth_actual'] = pd.to_datetime(df_main.year_of_birth.astype(str) + '/' + df_main.month_of_birth.astype(str) + '/01')
    df_main['age_in_days_delta'] = pd.to_timedelta(df_main['age_in_days'],'d')
    df_main['onset_year'] = (df_main['date_of_birth_actual'] + df_main['age_in_days_delta']).dt.year
    return df_main

In [ ]:
Cases_Onset= add_onset_year(Cases_HT)


In [ ]:
Cases_Onset

In [ ]:
for col in Cases_Onset:
    print (col)

In [ ]:
#define cases as HT patients with the onset year before 2013 

In [ ]:
Cases_HT_beforeOnset2013= Cases_Onset.loc[Cases_Onset['onset_year'] <2013]
Cases_HT_beforeOnset2013

In [ ]:
Cases_HT_beforeOnset2013.to_pickle("./Cases_HT_beforeOnset2013.pkl")

In [ ]:
#check pkl file
unpickled_df = pd.read_pickle("./Cases_HT_beforeOnset2013.pkl")
unpickled_df

In [ ]:
#case prediction 2013 - 2018

In [ ]:
Cases_HT_2013_2018 = Cases_Onset.loc[Cases_Onset['onset_year'] >= 2013]
Cases_HT_2013_2018

In [ ]:
################